In [22]:
import pandas as pd # To manage data as data frames
import numpy as np # To manipulate data as arrays
#from sklearn.linear_model import LogisticRegression # Classification model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

import math
import pickle

In [23]:
# import regression module
#from sklearn.linear_model import LinearRegression

# split predictors and response
#X = df.drop(['mpg'], axis=1)
#y = df['mpg']

In [24]:
df_train = pd.read_csv("./data/df_train.csv")
df_train.head()

,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,...,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min,load_shortfall_3h
0,0,2015-01-01 03:00:00,0.666667,level_5,0.0,0.666667,74.333333,64.000000,0.0,1.000000,...,265.938000,281.013000,269.338615,269.338615,281.013000,269.338615,274.254667,265.938000,265.938000,6715.666667
1,1,2015-01-01 06:00:00,0.333333,level_10,0.0,1.666667,78.333333,64.666667,0.0,1.000000,...,266.386667,280.561667,270.376000,270.376000,280.561667,270.376000,274.945000,266.386667,266.386667,4171.666667
2,2,2015-01-01 09:00:00,1.000000,level_9,0.0,1.000000,71.333333,64.333333,0.0,1.000000,...,272.708667,281.583667,275.027229,275.027229,281.583667,275.027229,278.792000,272.708667,272.708667,4274.666667
3,3,2015-01-01 12:00:00,1.000000,level_8,0.0,1.000000,65.333333,56.333333,0.0,1.000000,...,281.895219,283.434104,281.135063,281.135063,283.434104,281.135063,285.394000,281.895219,281.895219,5075.666667
4,4,2015-01-01 15:00:00,1.000000,level_7,0.0,1.000000,59.000000,57.000000,2.0,0.333333,...,280.678437,284.213167,282.252063,282.252063,284.213167,282.252063,285.513719,280.678437,280.678437,6620.666667


In [25]:
df_train.shape

(8763, 49)

In [26]:
df_train_clean = df_train.copy()

In [27]:
df_train_clean['time'] = pd.to_datetime(df_train_clean['time'])

In [28]:
df_train_clean['Valencia_wind_deg'] = df_train_clean['Valencia_wind_deg'].str.extract('(\d+)')

In [29]:
df_train_clean['Valencia_wind_deg'] = pd.to_numeric(df_train_clean['Valencia_wind_deg'])

In [30]:
df_train_clean['Seville_pressure'] = df_train_clean['Seville_pressure'].str.extract('(\d+)')

In [31]:
df_train_clean['Seville_pressure'] = pd.to_numeric(df_train_clean['Seville_pressure'])

In [32]:
# Replace NaN with Mean for Valencia Pressure
mean_Valencia_pressure=df_train_clean.Valencia_pressure.mean()
df_train_clean.loc[df_train_clean.Valencia_pressure.isnull(),'Valencia_pressure']=mean_Valencia_pressure

In [33]:
df_train_clean = df_train_clean.drop(['Unnamed: 0'], axis = 1)

In [34]:
df_train_clean_no_time = df_train_clean.copy()

In [35]:
df_train_clean_no_time = df_train_clean_no_time.drop(['time'], axis = 1)

In [36]:
corr_matrix = df_train_clean_no_time.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
print(to_drop)

['Valencia_temp', 'Seville_temp', 'Valencia_temp_min', 'Barcelona_temp_max', 'Madrid_temp_max', 'Barcelona_temp', 'Bilbao_temp', 'Barcelona_temp_min', 'Bilbao_temp_max', 'Seville_temp_min', 'Madrid_temp', 'Madrid_temp_min']


In [37]:
df_train_clean_no_time.drop(to_drop, axis=1, inplace=True)

In [38]:
#df_train_clean_no_time.drop(['Valencia_temp', 'Seville_temp', 'Valencia_temp_min', 'Barcelona_temp_max', 'Madrid_temp_max', 'Barcelona_temp', 'Bilbao_temp', 'Barcelona_temp_min', 'Bilbao_temp_max', 'Seville_temp_min', 'Madrid_temp', 'Madrid_temp_min'], axis=1, inplace=True)

In [39]:
df_train_clean_no_time.head()

,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,Seville_clouds_all,Bilbao_wind_deg,...,Bilbao_pressure,Seville_weather_id,Valencia_pressure,Seville_temp_max,Madrid_pressure,Valencia_temp_max,Bilbao_weather_id,Valencia_humidity,Bilbao_temp_min,load_shortfall_3h
0,0.666667,5,0.0,0.666667,74.333333,64.000000,0.0,1.000000,0.0,223.333333,...,1035.000000,800.0,1002.666667,274.254667,971.333333,269.888000,800.0,75.666667,269.338615,6715.666667
1,0.333333,10,0.0,1.666667,78.333333,64.666667,0.0,1.000000,0.0,221.000000,...,1035.666667,800.0,1004.333333,274.945000,972.666667,271.728333,800.0,71.000000,270.376000,4171.666667
2,1.000000,9,0.0,1.000000,71.333333,64.333333,0.0,1.000000,0.0,214.333333,...,1036.000000,800.0,1005.333333,278.792000,974.000000,278.008667,800.0,65.666667,275.027229,4274.666667
3,1.000000,8,0.0,1.000000,65.333333,56.333333,0.0,1.000000,0.0,199.666667,...,1036.000000,800.0,1009.000000,285.394000,994.666667,284.899552,800.0,54.000000,281.135063,5075.666667
4,1.000000,7,0.0,1.000000,59.000000,57.000000,2.0,0.333333,0.0,185.000000,...,1035.333333,800.0,1012.051407,285.513719,1035.333333,283.015115,800.0,58.333333,282.252063,6620.666667


In [40]:
y = df_train_clean_no_time.load_shortfall_3h
X = df_train_clean_no_time.drop(['load_shortfall_3h'], axis=1)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

In [41]:
# create model object
lin_model = LinearRegression()

In [42]:
lin_model.fit(train_X, train_y)

LinearRegression()

In [43]:
y_pred = lin_model.predict(X)

In [44]:
model_save_path = "lin_model.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(lin_model, file)